## Melody Features

This notebook seeks to illustrate how to use the Melodic Feature Set. The feature set is accessed using the top level function `get_all_features`. This function computes a wide range of features on every melody found in the input directory, returning a single .csv file with all melodies and their features. 

In [ ]:
from melody_features import get_all_features
from melody_features.corpus import get_corpus_files

first_ten_essen = get_corpus_files('essen', max_files=10)

features = get_all_features(first_ten_essen)
features.head()


We can save the results of the feature calculations to a csv file using the below:

In [ ]:
features.to_csv('output.csv')

The feature set has a few customisable aspects that change the behaviour of some of the feature calculations. There is no requirement to customise this configuration, as sensible values recommended in the literature are supplied as defaults. However, for users seeking more control over the behaviour of FANTASTIC and IDyOM, the `Config` dataclass is provided:

In [ ]:
# Import the Config dataclasses
from melody_features.features import Config, IDyOMConfig, FantasticConfig

Once we import these dataclasses, we can begin to customise our configuration:

In [ ]:
# Initialise the config object with the relevant parameters
from melody_features import essen_corpus

config = Config(
    corpus=essen_corpus,
    idyom={"pitch": IDyOMConfig(
        target_viewpoints=["cpitch"],
        source_viewpoints=[("cpint", "cpintfref")],
        ppm_order=1,
        models=":both"
    )},
        fantastic=FantasticConfig(
        max_ngram_order=2,
        phrase_gap=1.5
    ),
    key_estimation="always_read_from_file",
    key_finding_algorithm="krumhansl_schmuckler"
)

The `corpus` parameter operates on different levels. If you wish to use the same corpus for both FANTASTIC and IDyOM, you need only set it in the top level of `Config()`; you then do not need to supply it to `IDyOMConfig` or `FantasticConfig`. 

If you want to use different corpora for each different toolbox, `IDyOMConfig` and `FantasticConfig` will override whatever is supplied in the top level of `Config`.

In [ ]:
# Initialise the config object with different corpora
different_corpus_config = Config(
    corpus=None, # will be overridden
    idyom={"pitch": IDyOMConfig(
        target_viewpoints=["cpitch"],
        source_viewpoints=[("cpint", "cpintfref")],
        ppm_order=1,
        models=":both",
        corpus=None
    )},
        fantastic=FantasticConfig(
        max_ngram_order=2,
        phrase_gap=1.5,
        corpus=essen_corpus
    ),
    key_estimation="always_read_from_file",
    key_finding_algorithm="krumhansl_schmuckler"
)

features_different_corpus = get_all_features(first_ten_essen, config=different_corpus_config)
features_different_corpus.head()

We can also supply multiple IDyOM configurations, allowing us to compute information content using different 'viewpoints' or corpora in one run of the feature set. This can be achieved like so:

In [ ]:
multi_idyom_config = Config(
    corpus=essen_corpus,
    idyom={"pitch": IDyOMConfig(
        target_viewpoints=["cpitch"],
        source_viewpoints=[("cpint", "cpintfref")],
        ppm_order=1,
        models=":both",
        corpus=essen_corpus
    ),
    "rhythm": IDyOMConfig(
        target_viewpoints=["onset"],
        source_viewpoints=["ioi"],
        ppm_order=1,
        models=":both",
        corpus=None
    )},
        fantastic=FantasticConfig(
        max_ngram_order=2,
        phrase_gap=1.5,
        corpus=None
    ),
    key_estimation="always_read_from_file",
    key_finding_algorithm="krumhansl_schmuckler"
)

In [ ]:
# Now we can get the different IDyOM features along with everything else
features_different_idyom = get_all_features(first_ten_essen, config=multi_idyom_config)
features_different_idyom.head()

As well as skipping corpus-dependent features, we can choose to skip IDyOM entirely if we like, as it can be quite time-consuming if you don't intend to use its output:

In [ ]:
features_no_idyom = get_all_features(first_ten_essen, skip_idyom=True)
features_no_idyom.head()

There are two other methods for `key_estimation`: `infer_if_necessary` and `always_infer`. These settings rely on `key_finding_algorithm` to estimate the key of the melody. Specifically, `infer_if_necessary` will attempt to read the key from the MIDI file, and if it is unable to detect key information, will estimate it using the method from `key_finding_algorithm`. `always_infer` will ignore any key information in the MIDI file and estimate it in the same fashion. 